# Visualizing Molecular Structures in VR

This notebook shows how to download protein structures from the Protein Data Bank and visualize them in VR.

In [ ]:
# Install required packages (run once)
# !pip install biopython numpy requests immersivepoints

In [ ]:
import numpy as np
import requests
from io import StringIO
try:
    from Bio.PDB import PDBParser
except ImportError:
    print("Installing biopython...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'biopython'])
    from Bio.PDB import PDBParser

import immersivepoints as ip

## Download Protein Structure from PDB

Choose a PDB ID:
- `1A3N` - Hemoglobin (oxygen transport)
- `1MSO` - Insulin (blood sugar regulation)  
- `1BNA` - DNA double helix
- `6VXX` - SARS-CoV-2 spike protein
- `2HHB` - Deoxyhemoglobin
- `1AKE` - Adenylate kinase (enzyme)

In [ ]:
def download_pdb(pdb_id):
    """Download PDB file from RCSB"""
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        raise ValueError(f"Could not download PDB {pdb_id}")

# Choose your protein
PDB_ID = "1A3N"  # Hemoglobin

print(f"Downloading {PDB_ID} from RCSB Protein Data Bank...")
pdb_data = download_pdb(PDB_ID)
print(f"Downloaded {len(pdb_data)} bytes")

## Parse PDB and Extract Atomic Coordinates

In [ ]:
def element_to_hue(element):
    """Map element to hue value (0-1)"""
    element_colors = {
        'C': 0.0,    # Carbon - gray/black
        'N': 0.6,    # Nitrogen - blue
        'O': 0.0,    # Oxygen - red
        'S': 0.15,   # Sulfur - yellow
        'P': 0.08,   # Phosphorus - orange
        'H': 0.5,    # Hydrogen - white (usually omitted)
        'FE': 0.95,  # Iron - brown/red
        'ZN': 0.5,   # Zinc - gray
        'MG': 0.33,  # Magnesium - green
    }
    return element_colors.get(element.upper(), 0.5)

def pdb_to_points(pdb_string):
    """Convert PDB file to point cloud (XYZI format)"""
    # Parse PDB
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', StringIO(pdb_string))
    
    # Extract all atoms
    points = []
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    coord = atom.get_coord()
                    element = atom.element
                    hue = element_to_hue(element)
                    points.append([coord[0], coord[1], coord[2], hue])
    
    points = np.array(points, dtype=np.float32)
    
    # Center the molecule
    points[:, :3] -= points[:, :3].mean(axis=0)
    
    # Scale to reasonable VR size (molecules are tiny!)
    points[:, :3] *= 0.1
    
    return points

print("Parsing PDB file and extracting atoms...")
points = pdb_to_points(pdb_data)
print(f"Extracted {len(points)} atoms")
print(f"Bounding box: X=[{points[:,0].min():.2f}, {points[:,0].max():.2f}] "
      f"Y=[{points[:,1].min():.2f}, {points[:,1].max():.2f}] "
      f"Z=[{points[:,2].min():.2f}, {points[:,2].max():.2f}]")

## Visualize Inline in Jupyter Notebook

In [ ]:
# Render inline - you can rotate and explore
ip.renderPoints(points, point_size=0.3, background_color=0x000000)

## Generate VR Link

Click the link below to view in VR on your headset!

In [ ]:
# Generate VR link
ip.showVR(points, point_size=0.3)

## Save for Later Upload

In [ ]:
# Save as binary file for upload to immersivepoints.com
output_file = f"{PDB_ID}_molecule.xyzi"
points.astype(np.float32).byteswap().tofile(output_file)
print(f"Saved to {output_file}")
print(f"File size: {len(points) * 16 / 1024:.1f} KB")
print(f"\nUpload this file at: https://immersivepoints.com/upload.html")

## Try Different Proteins

Change the `PDB_ID` variable above and re-run the cells to explore different molecules:

- **Antibodies**: `1IGT`, `1HZH`
- **Enzymes**: `1AKE`, `4HHB`  
- **DNA/RNA**: `1BNA`, `1EHZ`
- **Viral proteins**: `6VXX` (COVID spike), `1JSF` (influenza)
- **Large complexes**: `1AON` (ribosome subunit)

Browse more at https://www.rcsb.org/